<a href="https://colab.research.google.com/github/KeerthanaAjith/Machine-Learning-Assignments/blob/main/Lab_8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Keerthana Ajith
# BL.EN.U4CSE21098
# CSE B

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/Book1.csv')
df

,age,income,student,credit rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31 . . .40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes
5,>40,low,yes,excellent,no
6,31 . . .40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


In [ ]:
# A1.

data = [
    ['<=30', 'high', 'no', 'fair', 'no'],
    ['<=30', 'high', 'no', 'excellent', 'no'],
    ['31 . . .40', 'high', 'no', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'low', 'yes', 'excellent', 'no'],
    ['31 . . .40', 'low', 'yes', 'excellent', 'yes'],
    ['<=30', 'medium', 'no', 'fair', 'no'],
    ['<=30', 'low', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'yes', 'fair', 'yes'],
    ['<=30', 'medium', 'yes', 'excellent', 'yes'],
    ['31 . . .40', 'medium', 'no', 'excellent', 'yes'],
    ['31 . . .40', 'high', 'yes', 'fair', 'yes'],
    ['>40', 'medium', 'no', 'excellent', 'no']
]

buys_computer_col = [row[-1] for row in data]

total_inst = len(buys_computer_col)
buys_computer_no = buys_computer_col.count('no')
buys_computer_yes = buys_computer_col.count('yes')

prior_no = buys_computer_no / total_inst
prior_yes = buys_computer_yes / total_inst

print(f'Prior probability for "buys_computer = no": {prior_no:.3f}')
print(f'Prior probability for "buys_computer = yes": {prior_yes:.3f}')

Prior probability for "buys_computer = no": 0.357
Prior probability for "buys_computer = yes": 0.643


In [ ]:
# A2.

def calculate_conditional_probability(feature, label, df):
    conditional_probs = {}
    classes = df[label].unique()

    for c in classes:
        class_data = df[df[label] == c]
        conditional_probs[c] = class_data[feature].value_counts(normalize=True).to_dict()

    return conditional_probs

for feature in df.columns[:-1]:  # Exclude the target variable
    conditional_probs = calculate_conditional_probability(feature, 'buys_computer', df)

    print(f"Class-conditional densities for {feature}:")
    for label, probs in conditional_probs.items():
        print(f"P({feature} | buys_computer = {label}):")
        for value, prob in probs.items():
            print(f"  P({feature} = {value} | buys_computer = {label}): {prob:.4f}")
        print()

Class-conditional densities for age:
P(age | buys_computer = no):
  P(age = <=30 | buys_computer = no): 0.6000
  P(age = >40 | buys_computer = no): 0.4000

P(age | buys_computer = yes):
  P(age = 31 . . .40 | buys_computer = yes): 0.4444
  P(age = >40 | buys_computer = yes): 0.3333
  P(age = <=30 | buys_computer = yes): 0.2222

Class-conditional densities for income:
P(income | buys_computer = no):
  P(income = high | buys_computer = no): 0.4000
  P(income = medium | buys_computer = no): 0.4000
  P(income = low | buys_computer = no): 0.2000

P(income | buys_computer = yes):
  P(income = medium | buys_computer = yes): 0.4444
  P(income = low | buys_computer = yes): 0.3333
  P(income = high | buys_computer = yes): 0.2222

Class-conditional densities for student:
P(student | buys_computer = no):
  P(student = no | buys_computer = no): 0.8000
  P(student = yes | buys_computer = no): 0.2000

P(student | buys_computer = yes):
  P(student = yes | buys_computer = yes): 0.6667
  P(student = no 

In [ ]:
# A3.

from scipy.stats import chi2_contingency

contingency_tb = pd.crosstab(index=df['age'], columns=[df['income'], df['student'], df['credit rating']])

chi2, p, _, _ = chi2_contingency(contingency_tb)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

alpha = 0.05
print(f"Significance level: {alpha}")
print(f"Is the p-value less than alpha? {'Yes' if p < alpha else 'No'}")

if p < alpha:
    print("Is rejecting the null hypothesis. There is evidence of a significant association between the features.")
else:
    print("Is failing to reject the null hypothesis. There is no significant evidence of an association between the features.")

Chi-square statistic: 12.95
P-value: 0.6764100579553458
Significance level: 0.05
Is the p-value less than alpha? No
Is failing to reject the null hypothesis. There is no significant evidence of an association between the features.


In [ ]:
# A4.

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

label_encoder = LabelEncoder()
for column in df.columns:
    df[column] = label_encoder.fit_transform(df[column])

X = df.drop('buys_computer', axis=1)
y = df['buys_computer']
Tr_X, Te_X, Tr_y, Te_y = train_test_split(X, y, test_size=0.2, random_state=42)

mod = GaussianNB()
mod.fit(Tr_X, Tr_y)

predictions = mod.predict(Te_X)

accuracy = accuracy_score(Te_y, predictions)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(Te_y, predictions))

Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [1]:
# A5.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

df1 = pd.read_csv('/content/waterPotability_updated (1).csv')
print(df1.sample(20))

X = df1.drop('Potability', axis=1)
y = df1['Potability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_pred = nb_classifier.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(metrics.classification_report(y_test, y_pred))


             ph    Hardness        Solids  Chloramines     Sulfate  \
268    6.869639  251.293448  21728.821295     8.803175  279.776857   
108    6.098157  189.995846  27776.901052     6.443098  347.930400   
587    9.735889  209.761513  10897.150289     6.106567  358.193121   
2253   6.155249  159.519502  19781.610600    10.081610  310.140661   
3025   7.149125  202.582542  23158.389096     6.067812  302.002743   
1430   7.397562  217.936799  24502.155146     5.919099  333.775777   
3092   7.260320  194.247742   8127.071254     6.674032  394.555835   
2282  10.524602  256.172323  31965.364041     5.007003  288.565983   
1657   5.467401  197.028926  12883.163531     9.264883  347.421266   
339    7.543700  161.623142  19116.012084     8.493407  289.959953   
2628   4.811832  150.451280  27547.807056     7.569544  296.002194   
2420   7.080795  201.827848  35418.735040     8.255971  253.580234   
223    9.925041  238.713393  10704.603182     6.110019  351.597423   
1901   7.916670  201

In [5]:
# Other Classifiers used for the temporary Dataset

# 2.Decision Tree
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier


X = df1.drop('Potability', axis=1)  # Features
y = df1['Potability']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)  # 70% training and 30% testing

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

print("Precision:", metrics.precision_score(y_test, y_pred))

print("Recall:", metrics.recall_score(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.5666327568667345
Precision: 0.4461942257217848
Recall: 0.44155844155844154
              precision    recall  f1-score   support

           0       0.64      0.65      0.65       598
           1       0.45      0.44      0.44       385

    accuracy                           0.57       983
   macro avg       0.54      0.54      0.54       983
weighted avg       0.57      0.57      0.57       983



In [6]:
# 3.Random Forest
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

print("Precision:", metrics.precision_score(y_test, y_pred))

print("Recall:", metrics.recall_score(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.671414038657172
Precision: 0.6581632653061225
Recall: 0.33506493506493507
              precision    recall  f1-score   support

           0       0.67      0.89      0.77       598
           1       0.66      0.34      0.44       385

    accuracy                           0.67       983
   macro avg       0.67      0.61      0.61       983
weighted avg       0.67      0.67      0.64       983

